In [1]:
%load_ext autoreload
%autoreload 2

Based on 



1.   https://towardsdatascience.com/hugging-face-transformers-fine-tuning-distilbert-for-binary-classification-tasks-490f1d192379
2.   https://www.analyticsvidhya.com/blog/2020/07/transfer-learning-for-nlp-fine-tuning-bert-for-text-classification/
3.   https://huggingface.co/transformers/training.html#fine-tuning-with-keras




**problem statement:**


*   a developer has to inspect an **artifact X**
*   Within the artifact, only a portion of the text is relevant to **input task Y**
*   We ought to build a model that establishes relationships between **Y** and **sentences x ∈ X** 
*  The model must determine: **is x relevant to task Y**




<br>

___

*Example of a task and an annotated artifact:*

<br>

[<img src="https://i.imgur.com/Zj1317H.jpg">](https://i.imgur.com/Zj1317H.jpg)




* The coloured sentences are sentences annotated as relevant to the input task. 
* The warmer the color, the more annotators selected that portion of the text. 
* For simplicity, we process the data and used sentences 

<br>

___

*Ultimately, our data is a tuple representing:*


*   **text** = artifact sentence

*   **question** = task description

*   **source** = URL of the artifact

*   **category_index** = whether sentence is relevant [or not] for the input task

*   **weights** = number of participants who annotated sentence as relevant


<br>

___



In [2]:
# @title Import data as JSON
import itertools
import json
import logging
import os
import sys
import random
from pathlib import Path

from Levenshtein import ratio
from colorama import Fore, Style

logger = logging.getLogger()
logger.level = logging.DEBUG
stream_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(stream_handler)

from ds_android import get_input_for_BERT
from data import greedy_stack_overflow_selection

raw_data = get_input_for_BERT()

print('Sample entry from data:')
print(json.dumps(raw_data[0], indent=4, sort_keys=True))

39 129  https://developer.android.com/training/permissions/requesting
14 21  https://stackoverflow.com/questions/5233543
4 34  https://github.com/morenoh149/react-native-contacts/issues/516
27 63  https://guides.codepath.com/android/Understanding-App-Permissions
9 161  https://www.avg.com/en/signal/guide-to-android-app-permissions-how-to-use-them-smartly
9 15  https://developer.android.com/training/volley/request
14 65  https://stackoverflow.com/questions/28504524
20 59  https://medium.com/@JasonCromer/android-asynctask-http-request-tutorial-6b429d833e28
5 97  https://www.twilio.com/blog/5-ways-to-make-http-requests-in-java
4 12  https://stackoverflow.com/questions/33241952
6 33  https://github.com/realm/realm-java/issues/776
3 17  https://stackoverflow.com/questions/8712652
8 59  https://dzone.com/articles/android-rotate-and-scale
5 470  https://developer.android.com/reference/android/widget/TextView
7 11  https://stackoverflow.com/questions/19025301
8 95  https://docs.oracle.com/java

4 54  https://developer.android.com/training/gestures/scroll
4 16  https://stackoverflow.com/questions/39588322
20 196  https://developer.android.com/training/dependency-injection/dagger-android
6 44  https://stackoverflow.com/questions/57235136
24 121  https://guides.codepath.com/android/dependency-injection-with-dagger-2
Sample entry from data:
{
    "category_index": 1,
    "question": "Permission Denial when trying to access contacts in Android",
    "source": "https://developer.android.com/training/permissions/requesting",
    "text": "Every Android app runs in a limited-access sandbox.",
    "weights": 1
}


In [3]:
# @title DICT that will store fold results
# If there is a previous execution for the same configuration, we load it from disk

# final results are the average of 3 distinct runs of this script.
# reason: avoid phishing results when BERT training procedures were exceptionally good
NUMBER_OF_RUNS = 1
# config_output = 'output/bert_ds_android_base.json'
config_output = 'output/bert_ds_android_fe.json' # for frame-elements filter
# config_output = 'output/bert_ds_android_fa.json' # for frame-association filters

fold_results = dict()

if os.path.isfile(config_output):
    logger.info(Fore.YELLOW + "Loading data from cache" + Style.RESET_ALL)
    with open(config_output) as input_file:
        fold_results = json.load(input_file)
        
if 'venn_diagram_set' not in fold_results:
    fold_results['venn_diagram_set'] = []     

Loading data from cache


In [4]:
# @title Set environment variables

import os
import contextlib
import tensorflow as tf
import os
import codecs
import numpy as np
import math
import json

import numpy as np
import pandas as pd

from collections import defaultdict, Counter
from tqdm import tqdm

USE_TPU = False
os.environ['TF_KERAS'] = '1'

# @title Initialize TPU Strategy
if USE_TPU:
    TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
    tf.contrib.distribute.initialize_tpu_system(resolver)
    strategy = tf.contrib.distribute.TPUStrategy(resolver)
    
from TFBertForTaskTextClassification import TFBertForTaskTextClassification
from TFBertForTaskTextClassification import TFBertForAndroidTaskTextClassification
from TFBertForTaskTextClassification import TFBertForSyntheticTaskTextClassification 

from metrics import MetricsAggregator

Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.


# BERT

In [5]:
# Init the model
model = TFBertForAndroidTaskTextClassification(model_id = 'bert-base-uncased')

# Configure filters. All other values are as default
model.target_output = 10
model.use_frame_filtering = False
model.match_frame_from_task = False
model.n_undersampling = 4
        
# Load tokenizer
model.tokenizer(cache_dir='/home/msarthur/scratch', local_files_only=True)

In [6]:
# base + frame-elements
model.use_frame_filtering = True
model.match_frame_from_task = False

In [7]:
# # base + frame-associations
# model.use_frame_filtering = False
# model.match_frame_from_task = True

In [8]:
# @title 10-fold cross validation WIP
CORPUS = raw_data

all_tasks = sorted(list(set([d['question'] for d in raw_data])))
rseed = 20210343
random.seed(rseed)
random.shuffle(all_tasks)

from sklearn.model_selection import KFold


file_handler = logging.FileHandler('/home/msarthur/scratch/LOG-bert_ds_android.ans')
file_handler.setLevel(logging.DEBUG)
logger.addHandler(file_handler)


n_splits = 10
kf = KFold(n_splits=n_splits, random_state=rseed)
np_tasks_arr = np.array(all_tasks)


idx_split = 0
# FIXME: folds are actually the total number of runs
for train_index, test_index in kf.split(np_tasks_arr):

    idx_split = str(idx_split)
    eval_fold = True
    # 10 runs per fold to avoid reporting peek results in a given fold
    if idx_split in fold_results and fold_results[idx_split]['run_cnt'] >= NUMBER_OF_RUNS:
        logger.info(Fore.RED + f"Fold {idx_split} FULLY TESTED" + Style.RESET_ALL)
        eval_fold = False


    if eval_fold:
        model.metrics.reset_aggregators()

#         test_tasks_lst = np_tasks_arr[test_index].tolist()
        test_tasks_lst = greedy_stack_overflow_selection(raw_data)

        logger.info("")
        logger.info(Fore.RED + f"Fold {idx_split}" + Style.RESET_ALL)
        logger.info('\n'.join(test_tasks_lst))

        # <------------------------------------------------------------------------- INPUT
        df_train, df_val, df_test, weights = model.get_train_val_test(
            CORPUS, test_tasks_lst
        )
        

        logger.info('-' * 10)
        logger.info(Fore.RED + 'train'+ Style.RESET_ALL)
        logger.info(str(df_train.category_index.value_counts()))
        logger.info("")

        logger.info(Fore.RED + 'test'+ Style.RESET_ALL)
        logger.info(str(df_test.category_index.value_counts()))
        logger.info("")

        logger.info(Fore.RED + 'weights'+ Style.RESET_ALL)
        logger.info(str(weights))
        logger.info('-' * 10)
        
        # <------------------------------------------------------------------------- TRAIN

        # Encode X_train
        train_encodings = model.encode(df_train)
        train_labels = df_train['category_index'].tolist()

        # Encode X_valid
        val_encodings = model.encode(df_val)
        val_labels = df_val['category_index'].tolist()


        # https://huggingface.co/transformers/custom_datasets.html
        train_dataset = tf.data.Dataset.from_tensor_slices((
            dict(train_encodings),
            train_labels
        ))

        val_dataset = tf.data.Dataset.from_tensor_slices((
            dict(val_encodings),
            val_labels
        ))


        fine_tunned_keras_model = model.build(
            train_dataset, val_dataset, weights, 
            checkpoint_filepath='/home/msarthur/scratch/best_model', 
            cache_dir='/home/msarthur/scratch', 
            local_files_only=True
        )
        
        
        if model.match_frame_from_task:
            __frame_pairs = model.fn_frame_pairs.get_most_common_frame_relationships(df_train)
            model.sentence_task_frame_pairs = __frame_pairs

        # <------------------------------------------------------------------------- TEST
        logger.info("")
        logger.info(Fore.RED + f"Testing model" + Style.RESET_ALL)
        for source in df_test["source"].unique():
            df_source = df_test[df_test["source"] == source]   
            logger.info(source)
            
            model.test(source, df_source, fine_tunned_keras_model)
                        

        # <------------------------------------------------------------------------- METRICS   
        
        prediction_metrics, api_metrics, so_metrics, git_metrics, misc_metrics = model.get_evaluation_metrics()
        
        MetricsAggregator.add_idx_fold_results(
            idx_split, fold_results, prediction_metrics,
            api_metrics, so_metrics, git_metrics, misc_metrics
        )

        fold_results['venn_diagram_set'] += model.metrics.venn_diagram_set
        fold_results['venn_diagram_set'] = list(set(fold_results['venn_diagram_set']))


        _precision, _recall, _f1score = MetricsAggregator.avg_macro_metric_for(prediction_metrics)

        logger.info("")
        logger.info(Fore.YELLOW + "Model metrics" + Style.RESET_ALL)
        logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
        logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
        logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)


    idx_split = int(idx_split)
    idx_split += 1
    logger.info(f"next {idx_split}")
#     if idx_split >= 6:
#         break
    


Fold 0
Don’t leak MockWebServer ports across tests
Java: Efficient ArrayList filtering?
Android PDF Rendering
Is there an accepted best-practice on making asynchronous HTTP requests in Android?
Support for GoogleApiClient and new FusedLocationProviderApi
Doesn't scroll properly inside ViewPager
Quick Actions don't get displayed on Android 7.0
Dagger 2 doesn't implement some of the component methods in Android project with custom annotation processor
Explanation of the getView() method of an ArrayAdapter
Custom Annotations in Retrofit 2.0
Hide MarkerView when nothing selected
No lock screen controls ever
Permission Denial when trying to access contacts in Android
Android App Retrieve Data from Server but in a Secure way
Android Gallery with pinch zoom
Wait for 2 async REST calls to result in success or error
Application icon doesn&#39;t show up in Android action bar
Write and Read a json data to internal storage android
Not receiving notifications when phone is locked and connected thr

/home/msarthur/hface/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
100%|██████████| 7908/7908 [00:00<00:00, 731826.14it/s]

----------
train
0    482
1    121
Name: category_index, dtype: int64

test
0    3740
1     172
Name: category_index, dtype: int64

weights
{0: 1.0, 1: 4.0}
----------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x2b7890fcb3d0>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expe

...

Epoch 1/10
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
10/10 [==============================] - ETA: 0s - loss: 1.0766 - sparse_categorical_accuracy: 0.4362The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output

Precision: 0.5967
Recall: 0.8080
F1: 0.6393
2 entries logged
https://developer.android.com/reference/android/graphics/pdf/PdfRenderer
--------------------
Y
[0s] 36 [1s] 8
predicted
[0s] 34 [1s] 10
--------------------
Accuracy: 0.7727
macro_f1: 0.6508
Precision: 0.6412
Recall: 0.6667
F1: 0.6508
4 entries logged
https://developer.android.com/guide/topics/providers/content-provider-creating
https://developer.android.com/reference/android/widget/ArrayAdapter
--------------------
Y
[0s] 44 [1s] 3
predicted
[0s] 37 [1s] 10
--------------------
Accuracy: 0.7660
macro_f1: 0.5090
Precision: 0.5230
Recall: 0.5644
F1: 0.5090
1 entries logged
https://www.raywenderlich.com/10091980-testing-rest-apis-using-mockwebserver
https://www.twilio.com/blog/asynchronous-api-requests-java-completablefutures
--------------------
Y
[0s] 48 [1s] 2
predicted
[0s] 40 [1s] 10
--------------------
Accuracy: 0.7600
macro_f1: 0.4318
Precision: 0.4750
Recall: 0.3958
F1: 0.4318
0 entries logged
https://stackoverflow.co

/home/msarthur/hface/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


--------------------
Y
[0s] 13 [1s] 2
predicted
[0s] 6 [1s] 9
--------------------
Accuracy: 0.4000
macro_f1: 0.3541
Precision: 0.4722
Recall: 0.4423
F1: 0.3541
1 entries logged
https://stackoverflow.com/questions/40168601
--------------------
Y
[0s] 2 [1s] 2
predicted
[0s] 2 [1s] 2
--------------------
Accuracy: 0.5000
macro_f1: 0.5000
Precision: 0.5000
Recall: 0.5000
F1: 0.5000
1 entries logged
https://stackoverflow.com/questions/38980595
--------------------
Y
[0s] 3 [1s] 2
predicted
[0s] 0 [1s] 5
--------------------
Accuracy: 0.4000
macro_f1: 0.2857
Precision: 0.2000
Recall: 0.5000
F1: 0.2857
2 entries logged

Model metrics
precision: 0.523
recall:    0.586
f1-score:  0.519
next 10


/home/msarthur/hface/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/msarthur/hface/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [16]:
# with open(config_output, 'w') as outfile:
#     json.dump(fold_results, outfile, sort_keys=True, indent=4)
#     logger.info(Fore.RED + "Output successfully saved to: {}".format(config_output) + Style.RESET_ALL)

Output successfully saved to: output/bert_ds_android_fe.json


## Results

In [10]:
__precision, __recall, __fscore = MetricsAggregator.get_full_exec_results(fold_results)

logger.info("\n")
logger.info(Fore.RED + "AGGREGATED METRICS" + Style.RESET_ALL)
logger.info("\nprecision: " + Fore.RED + "{:.3f}".format(np.mean(__precision)) + Style.RESET_ALL)
logger.info("recall:    " + Fore.RED + "{:.3f}".format(np.mean(__recall)) + Style.RESET_ALL)
logger.info("f1-score:  " +  Fore.RED + "{:.3f}".format(np.mean(__fscore)) + Style.RESET_ALL)



AGGREGATED METRICS

precision: 0.525
recall:    0.563
f1-score:  0.514


In [11]:
if np.mean(__precision) >= 0.54:
    with open(config_output, 'w') as outfile:
        json.dump(fold_results, outfile, sort_keys=True, indent=4)
        logger.info(Fore.RED + "Output successfully saved to: {}".format(config_output) + Style.RESET_ALL)

In [12]:
# with open(config_output, 'w') as outfile:
#     json.dump(fold_results, outfile, sort_keys=True, indent=4)
#     logger.info(Fore.RED + "Output successfully saved to: {}".format(config_output) + Style.RESET_ALL)

## Stack Overflow results

In [13]:
__precision, __recall, __fscore = MetricsAggregator.get_full_exec_results(fold_results, result_type="so")

logger.info("\n")
logger.info(Fore.RED + "Stack Overflow metrics" + Style.RESET_ALL)
logger.info("\nprecision: " + Fore.RED + "{:.3f}".format(np.mean(__precision)) + Style.RESET_ALL)
logger.info("recall:    " + Fore.RED + "{:.3f}".format(np.mean(__recall)) + Style.RESET_ALL)
logger.info("f1-score:  " +  Fore.RED + "{:.3f}".format(np.mean(__fscore)) + Style.RESET_ALL)




Stack Overflow metrics

precision: 0.533
recall:    0.588
f1-score:  0.524


## Examples of text retrieved

In [14]:
#@title Sample prediction outputs for API sources

logger.info(Fore.RED + "API" + Style.RESET_ALL)
model.metrics.examples_per_source_type(source_type='api', n_samples=8)

API

How to record phone calls in Android
https://developer.android.com/guide/topics/media/mediarecorder

[w=0][y=1][p=0.5890] Note: The Android Emulator can not record audio.

[w=1][y=1][p=0.5659] Starting with Android 8.0 ( API level 26 ) you can use a MediaMuxer to record multiple simultaneous audio and video streams.

[w=0][y=1][p=0.5236] To record raw audio select UNPROCESSED.

[w=0][y=1][p=0.5148] Note that starting with Android 8.0 ( API level 26 ) MediaRecorder supports the MPEG2_TS format, which is useful for streaming: Kotlin Java

[w=0][y=1][p=0.3432] The timestamp must be in the same time base as the video and audio tracks.

[w=0][y=1][p=0.3265] These pages cover topics relating to recording, storing, and playing back audio and video.

[w=0][y=1][p=0.1907] When you use a MediaExtractor to extract a file that contains metadata tracks, the metadata's mime format appears as an instance of MediaFormat.

[w=0][y=1][p=0.1245] The sample code below shows how to implement this beha


[w=0][y=1][p=0.0655] Writing metadata is the same as writing video or audio data, except that the data does not come from a MediaCodec.

[w=0][y=1][p=0.0641] The timestamp must be in the same time base as the video and audio tracks.

[w=0][y=1][p=0.0533] You can also add one or more metadata tracks with custom information for each frame, but only to MP4 containers.

[w=1][y=1][p=0.8771] Use the addTrack ( ) method to mix multipe tracks together.

[w=2][y=1][p=0.2131] This document shows you how to use MediaRecorder to write an application that captures audio from a device microphone, save the audio, and play it back ( with MediaPlayer ).

[w=0][y=1][p=0.1704] Some devices do not support unprocessed input.

[w=0][y=1][p=0.1566] If it is not, try using VOICE_RECOGNITION instead, which does not employ AGC or noise suppression.

[w=0][y=1][p=0.1494] Note: The Android Emulator can not record audio.

[w=0][y=1][p=0.1473] Be sure to test your code on a real device that can record.

[w=0][y=1


[w=0][y=1][p=0.1079] In some cases calling a newly-introduced API results in a no-op on older devices.

[w=1][y=1][p=0.7929] Beginning with Android 5.0, notifications can appear on the lock screen.

[w=0][y=1][p=0.7895] On Android 7.1 ( API level 25 ) and below, importance of each notification is determined by the notification's priority.

[w=0][y=1][p=0.7648] On Android 8.0 ( API level 26 ) and above, users can additionally allow notifications through for app-specific categories ( also known as channels ) by overriding Do Not Disturb on a channel-by-channel basis.

[w=0][y=1][p=0.7595] The channel is also where you specify the importance level for your notifications on Android 8.0 and higher.

[w=0][y=1][p=0.7150] If the user has a paired Wear OS device, all your notifications appear there automatically, including expandable detail and action buttons.

[w=0][y=1][p=0.6173] Some details of the notification appear only in the expanded view.

[w=0][y=1][p=0.5261] Additionally, see the A


[w=0][y=1][p=0.6579] To avoid bombarding your users with multiple or redundant notifications when you have additional updates, you should consider updating an existing notification rather than issuing a new one, or consider using the inbox-style notification to show conversation updates.

[w=0][y=1][p=0.6566] Apps with active notifications display a notification `` badge'' on top of their app icon on the home/launcher screen.

[w=0][y=1][p=0.6124] Users can tap the notification to open your app or take an action directly from the notification.

[w=0][y=1][p=0.5868] Additionally, see the Android Notifications Sample for a demonstration of best practices in using the Notification.Style API in both mobile and wearable apps.

[w=0][y=1][p=0.5342] Users can then dismiss or act on notifications from that menu, similar to the notification drawer.

[w=0][y=1][p=0.4936] Users can drag down on a notification in the drawer to reveal the expanded view, which shows additional content and action bu


[w=0][y=1][p=0.7961] Send alerts to owners: Check this checkbox if you want to receive emails about the reCAPTCHA API.

[w=2][y=1][p=0.1681] The following code snippet shows how to invoke this method:

[w=2][y=1][p=0.0988] Pass in your API site key as a parameter.

[w=1][y=1][p=0.0964] You still need to validate the user's response token from your backend server.

[w=1][y=1][p=0.0783] To learn how to validate the user's response token, see Verifying the user's response.

[w=0][y=1][p=0.0772] In order for an app to use the API, the package name that you enter must be an exact match of the package name for that app.

[w=0][y=1][p=0.0757] For more information, see Set Up Google Play Services.

[w=1][y=1][p=0.0747] You use the site key when you send the verify request, and you use the secret key when you validate the user response token.

[w=0][y=1][p=0.0747] The calling app's package name doesn't match any of the names that you've associated with the site key.

[w=0][y=1][p=0.0735] If th

...


[w=0][y=1][p=0.2664] JsonObjectRequest and JsonArrayRequest ( both subclasses of JsonRequest ).

[w=3][y=1][p=0.1626] For example, this snippet fetches a JSON feed and displays it as text in the UI:

[w=0][y=1][p=0.5481] This lesson describes how to use the common request types that Volley supports:

[w=3][y=1][p=0.5418] For example, this snippet fetches a JSON feed and displays it as text in the UI:

[w=0][y=1][p=0.3931] For information on how to implement your own custom request, see Implementing a Custom Request.

[w=0][y=1][p=0.3736] See Setting Up a Request Queue for an example.

[w=0][y=1][p=0.1638] This lesson describes how to use these standard request types.

[w=1][y=1][p=0.1492] Both classes are based on the common base class JsonRequest.

[w=1][y=1][p=0.1349] JsonObjectRequest -- A request for retrieving a JSONObject response body at a given URL, allowing for an optional JSONObject to be passed in as part of the request body.

[w=1][y=1][p=0.0729] JsonArrayRequest -- A requ


--------------------

Different actions from contact info depending on whether hitting back key or back arrow in top left
https://developer.android.com/guide/navigation/navigation-custom-back

[w=3][y=1][p=0.4393] For example, if you added three callbacks named one, two and three in order, they would be invoked in the order of three, two, and one, respectively.

[w=1][y=1][p=0.4146] The OnBackPressedDispatcher controls how Back button events are dispatched to one or more OnBackPressedCallback objects.

[w=0][y=1][p=0.3967] This means that in the preceding example, callback two would be invoked only if callback three was not enabled.

[w=3][y=1][p=0.2841] You can provide multiple callbacks via addCallback ( ).

[w=2][y=1][p=0.4119] All Android devices provide a Back button for this type of navigation, so you should not add a Back button to your app's UI.

[w=0][y=1][p=0.3766] 

[w=1][y=1][p=0.2456] Callbacks are added via the addCallback methods.

[w=1][y=1][p=0.2332] Only when a callb


[w=0][y=1][p=0.1358] This usually allows Android to properly navigate to previous destinations when the Back button is pressed.

[w=1][y=1][p=0.1214] Only when a callback is enabled ( i.e., isEnabled ( ) returns true ) will the dispatcher call the callback's handleOnBackPressed ( ) to handle the Back button event.

[w=0][y=1][p=0.8826] However, in cases where you want to remove the OnBackPressedCallback entirely, you should call remove ( ).

[w=0][y=1][p=0.8747] Changing the enabled state on the OnBackPressedCallback is strongly recommended for temporary changes as it maintains the ordering described above, which is particularly important if you have callbacks registered on multiple different nested lifecycle owners.

[w=1][y=1][p=0.8714] If you are using onBackPressed ( ) to handle Back button events, we recommend using a OnBackPressedCallback instead.

[w=3][y=1][p=0.8528] For example, if you added three callbacks named one, two and three in order, they would be invoked in the order


[w=0][y=1][p=0.6711] From this screen, users can prevent the system from resetting your app's permissions by doing the following:

[w=0][y=1][p=0.6315] Tap Permissions, which loads the App permissions settings screen.

[w=3][y=1][p=0.6273] Check whether the user has already granted the runtime permission that your app requires.

[w=0][y=1][p=0.5568] If the user selects this option in the dialog, your app is granted a temporary one-time permission.

[w=0][y=1][p=0.4350] Verify that your app can handle the auto-reset event.

[w=0][y=1][p=0.1042] If this method returns true, show an educational UI to the user.

[w=0][y=1][p=0.0956] The following steps show how to use the RequestPermission contract.

[w=0][y=1][p=0.7951] If the user revokes the one-time permission, such as in system settings, your app can not access the data, regardless of whether you launched a foreground service.

[w=0][y=1][p=0.7734] Users see a system permission dialog, where they can choose whether to grant a particu


[w=0][y=1][p=0.1574] To direct the user to your app's page in system settings, invoke an intent that includes the Intent.ACTION _ AUTO_REVOKE_PERMISSIONS intent action.

[w=3][y=1][p=0.1511] Check whether the user has already granted the runtime permission that your app requires.

[w=0][y=1][p=0.6733] If the user revokes the one-time permission, such as in system settings, your app can not access the data, regardless of whether you launched a foreground service.

[w=1][y=1][p=0.6666] This method returns either PERMISSION_GRANTED or PERMISSION_DENIED, depending on whether your app has the permission.

[w=1][y=1][p=0.6646] If the user denied the permission instead, gracefully degrade your app experience so that it provides functionality to the user, even without the information that's protected by that permission.

[w=1][y=1][p=0.6643] If the user denies or revokes a permission that a feature needs, gracefully degrade your app so that the user can continue using your app, possibly by di

In [15]:
#@title Sample prediction outputs for SO sources

logger.info(Fore.RED + "SO" + Style.RESET_ALL)
model.metrics.examples_per_source_type(source_type='so', n_samples=8)

SO

Custom Annotations in Retrofit 2.0
https://stackoverflow.com/questions/47760861

[w=0][y=1][p=0.6188] While if you do not insist on custom annotations, the easiest way for now in my opinion is to add custom header to the api interface, then read and remove it in the interceptor.

[w=1][y=1][p=0.6156] Retrofit Changelog:

[w=0][y=1][p=0.6119] What you need to do is have two OkHttp clients, with their respective instances of Retrofit.

[w=0][y=1][p=0.6021] I have similar requirement, what I found is Annotation can be read in LINK, LINK and LINK.

[w=0][y=1][p=0.5897] fun < T: Annotation > Request.getCustomAnnotation -LRB- annotationClass: Class <T> -RRB-: T ?

[w=0][y=1][p=0.5777] request.getCustomAnnotation -LRB- YourAnnotation:: class.java -RRB-

[w=0][y=1][p=0.4784] I'm using as extension function:

[w=0][y=1][p=0.4596] These can be read in CallAdapters or OkHttp Interceptors for tracing, analytics, varying behavior, and more.

[w=0][y=1][p=0.3785] method -LRB- -RRB- ?

[w=0][y=1]


[w=2][y=1][p=0.8719] Since the version of Retrofit 2.6.0, you can get the annotations in OkHttp Interceptor using the tag field like this:

[w=0][y=1][p=0.8584] It uses a custom CallAdapter to get annotation @Authenticated, and put data into a map, which later parsed in the Interceptor.

[w=0][y=1][p=0.8567] What you need to do is have two OkHttp clients, with their respective instances of Retrofit.

[w=2][y=1][p=0.8304] Then inside of the interceptor, you can verify if the request is annotated or no.

[w=0][y=1][p=0.3362] I'm using as extension function:

[w=0][y=1][p=0.2929] And you can use then like that:

[w=0][y=1][p=0.1430] .

[w=0][y=1][p=0.1430] .

[w=0][y=1][p=0.1162] I also found two articles as examples for implementation on each way.

[w=0][y=1][p=0.1045] Did try to comment there but it looks too ugly because don't know how to make multiline.

[w=0][y=1][p=0.3378] In GsonCacheableConverter, it overrides responseBodyConverter -LRB- -RRB- to persist response tagged with @Cac


[w=0][y=1][p=0.0809] Activity code:

[w=0][y=1][p=0.3226] The accepted answer resulted in my fragments now being visible.

[w=0][y=1][p=0.1818] No need to use ScrollView and NestedScroolview Just use SliderLayout it will work perfectly.

[w=0][y=1][p=0.1389] add below dependency inside build.gradle -LRB- app level -RRB-

[w=1][y=1][p=0.1383] Here is full code for my CustomViewPager:

[w=0][y=1][p=0.1334] So let me do it for you: the correct hierarchy as suggested by google blogspot it is working fine for me: Check this out

[w=2][y=1][p=0.5717] Use android: fillViewport = `` true'' in ScrollView

[w=0][y=1][p=0.4870] However, it left me with the problem that my NestedScrollView would no longer scroll !

[w=0][y=1][p=0.3160] -LRB- I also removed the new android: fillViewport = `` true'' attribute from my NestedScrollView element as it seems this is now no longer needed. -RRB-

[w=0][y=1][p=0.2467] add below dependency inside build.gradle -LRB- app level -RRB-

[w=0][y=1][p=0.1923] layo


[w=0][y=1][p=0.4457] note that the 2 null args may allow you to clip some portion in the page and perform a transformation -LRB- using a LINK -RRB- of the clip

[w=3][y=1][p=0.3087] Android-Lollipop -LRB- api 21 -RRB- introduce a new API: LINK

[w=0][y=1][p=0.1735] LINK: LINK

[w=0][y=1][p=0.1442] This library is simple and works well: LINK LINK

[w=0][y=1][p=0.1390] Some phones -LRB- like the Nexus One -RRB- come with a version of LINK pre-installed so it may be as easy as sending the appropriate Intent once you've saved the file to the SD card.

[w=0][y=1][p=0.1266] Shortly:

[w=0][y=1][p=0.0861] I assure you that it works !

[w=0][y=1][p=0.9057] To open a pdf from a byte array, you can use RadaeePDF, you can do the following into your activity:

[w=0][y=1][p=0.8552] Some phones -LRB- like the Nexus One -RRB- come with a version of LINK pre-installed so it may be as easy as sending the appropriate Intent once you've saved the file to the SD card.

[w=0][y=1][p=0.3877] One shortcomin


[w=0][y=1][p=0.4992] instead of:

[w=1][y=1][p=0.4881] As you can read LINK:

[w=3][y=1][p=0.4427] If you would like an application icon -LRB- but I discourage it -RRB-, you can use the method setLogo -LRB- -RRB-.

[w=0][y=1][p=0.1397] and not:

[w=0][y=1][p=0.1035] And finally:

[w=0][y=1][p=0.1016] and then:

[w=0][y=1][p=0.0918] Reference: LINK

[w=3][y=1][p=0.8797] A Toolbar is a generalization of action bars for use within application layouts.

[w=0][y=1][p=0.2317] actionBar = getActionBar -LRB- -RRB- ;

[w=1][y=1][p=0.1242] As you can read LINK:

[w=1][y=1][p=0.1205] You are using the AppCompat version 21 + and it is normal.

[w=0][y=1][p=0.1123] and not:

[w=0][y=1][p=0.0844] In activity add this and try

[w=0][y=1][p=0.0830] Reference: LINK

[w=0][y=1][p=0.0784] Then in the onCreate method insert the following lines:

[w=0][y=1][p=0.9136] In modern Android UIs developers should lean more on a visually distinct color scheme for toolbars than on their application icon.

[w=3][y=


[w=2][y=1][p=0.8628] After that override getView -LRB- -RRB- method and make sure to return your custom View there.

[w=0][y=1][p=0.8623] So all you need to implement is getView

[w=0][y=1][p=0.8620] you need to override getCount method of the BaseAdapter to return total number of views to diplay.

[w=0][y=1][p=0.6499] And overrides getCount for you.

[w=3][y=1][p=0.1705] It triggers when you scroll the view -LRB- list for example -RRB-.

[w=2][y=1][p=0.0530] A reference to the parent view that this view will be a child of.

[w=0][y=1][p=0.0529] position:

[w=0][y=1][p=0.0526] In your case it should be a Layout with ImageView and TextView -LRB- and dont forget to fill them -RRB-.

[w=0][y=1][p=0.8178] And in getView you get following things:

[w=2][y=1][p=0.8061] After that override getView -LRB- -RRB- method and make sure to return your custom View there.

[w=0][y=1][p=0.7943] you need to override getCount method of the BaseAdapter to return total number of views to diplay.

[w=1][y=


[w=2][y=1][p=0.8149] Javac annotation processor uses rounds instead of defining processors order.

[w=0][y=1][p=0.7556] - proc: only means that only annotation processing is done, without any subsequent compilation.

[w=0][y=1][p=0.7514] Dagger annotation processor will process the TypeElement annotated with @dagger.

[w=0][y=1][p=0.7290] Sometimes it makes sense to move annotation processing to a difference gradle module, but I don't this that this is what you want.

[w=0][y=1][p=0.5822] In your Gradle build script, you need to define the preprocessing task and add a dependency on your task to the javaCompile task.

[w=0][y=1][p=0.3608] So, back to your question.

[w=0][y=1][p=0.1737] LINK

[w=0][y=1][p=0.1469] Explanation:

[w=0][y=1][p=0.1386] Solution:

[w=2][y=1][p=0.8003] But the easiest option is to generate java code directly and your generated java classes will be picked up by javac automatically, launching second round of annotation processing, where dagger will process them


[w=0][y=1][p=0.5990] after that, you can easily start recording anywhere you want

[w=2][y=1][p=0.5935] Here is a LINK on how to record audio using the LINK.

[w=2][y=1][p=0.5778] I am using mic to record phone audio and also use the Telephony manager to find the calling state.

[w=0][y=1][p=0.5041] and after finishing recording you can easily also stop the recording

[w=0][y=1][p=0.1272] As long as the audio source options work, you should be good to go.

[w=0][y=1][p=0.6165] to record just hit the menu button while in call in android phone it will store conversation in amr format and in root directory of sd card max 20 min conversation.

[w=0][y=1][p=0.5805] First off, you have to be careful with recording calls as there are legal requirements depending on the country.

[w=2][y=1][p=0.5643] Here is a LINK on how to record audio using the LINK.

[w=2][y=1][p=0.5301] I am using mic to record calls for better support and compatibility.

[w=2][y=1][p=0.5267] I am using mic to record pho


[w=0][y=1][p=0.6563] after that, you can easily start recording anywhere you want

[w=2][y=1][p=0.6233] I am using mic to record phone audio and also use the Telephony manager to find the calling state.

[w=0][y=1][p=0.5912] and after finishing recording you can easily also stop the recording

[w=0][y=1][p=0.1816] As long as the audio source options work, you should be good to go.

--------------------

Android App Retrieve Data from Server but in a Secure way
https://stackoverflow.com/questions/8184492

[w=0][y=1][p=0.5941] This link from Android developers site advice you some of the good tips for security - LINK

[w=0][y=1][p=0.5708] But still there are many vulnerabilities are possible by poor design of the apps.

[w=0][y=1][p=0.5372] To conclude, if your information is not super-duper sensitive -LRB- e.g. credit card information -RRB-, I'd suggest just sticking with the default security provided by Android -LRB- i.e. save everything in plain text, knowing other apps can't access 


[w=0][y=1][p=0.0908] You can't get a password that only exists in someone's -LRB- the user -RRB- head from decompiled code :-RRB-.

[w=0][y=1][p=0.0884] .

[w=0][y=1][p=0.0840] After enabling HTTPS, here are two extra security measures I would propose:

[w=2][y=1][p=0.0829] The second method is more useful if you are also doing plain HTTP requests, where everyone can see the parameters being sent.

[w=0][y=1][p=0.6631] It's not 100 % secure -LRB- a hacker could de-compile your app and figure out how to decrypt the data -RRB-, but it's a major pain to crack and will stop most hackers.

[w=0][y=1][p=0.6399] You can obfuscate as much as you want, if a hacker is really determined to get to your data, he will be able to so by decompiling your application and spending many sleepless nights passing through your code and figuring out how the requests are formed.

[w=0][y=1][p=0.4973] LINK http://developer.android.com/reference/javax/net/ssl/package-summary.html LINK

[w=0][y=1][p=0.3875] Howe